In [1]:
import numpy as np
from scipy import *
from pylab import *
import healpy as hp
import os
from healpy import pixelfunc
from aperture_mass_computer import aperture_mass_computer_curved_sky
from helpers_plot import initPlot, finalizePlot, prepareMap3Plot

# Measurements of $\langle M_\mathrm{ap}^3\rangle$ from curved sky fields
In this notebook we take curved sky fields from the simulations and measure the third-order aperture statistics using the kappa-maps

Author: Laila Linke

The Map3 measurement code is developed together with Sven Heydenreich, the Read-in of the simulations is based on the example notebook by Joachim Harnois-Déraps and Jia Liu

In [2]:
thetas=[4,8,16,32] # Aperture radii to use [arcmin]
Nthetas=len(thetas)
# Filenames

######## curved sky map
fn_map_curved = '/global/homes/j/jharno/IA-infusion/SkySim5000/kappa/1.0060kappa.npy'

######## shear maps
fn_gamma1="/global/homes/j/jharno/IA-infusion/SkySim5000/shear/1.0060gamma1.npy"
fn_gamma2="/global/homes/j/jharno/IA-infusion/SkySim5000/shear/1.0060gamma2.npy"

## Measurement from Kappa map

In [3]:
# Load Kappa Map
hpmap = np.load(fn_map_curved)

In [4]:
# Downgrade the kappa map
nside=2**11
hpmap_degrad=hp.ud_grade(hpmap, nside)

In [5]:
hp.nside2resol(nside, arcmin = True)  

0.42943580147717575

In [6]:
# Set masks (only use an octant)
mask=np.where(hpmap_degrad!=0)
outside=np.where(hpmap_degrad==0)

In [ ]:
# Calculate the Aperture Mass Maps from the kappa
Maps=[]
for i,theta in enumerate(thetas):
    ac=aperture_mass_computer_curved_sky(nside, theta)
    map=ac.Map_fft_from_kappa(hpmap_degrad)

    map[outside]=0
    Maps.append(map)


In [ ]:
# Plotting of Aperture Mass Maps
for i in range(Nthetas):
    map=Maps[i]
    hp.mollview(map, title=r"$\theta_\mathrm{ap}$="+f"{thetas[i]} arcmin", nest=False)
    hp.graticule()

In [ ]:
# Calculate third-order aperture statistics

Map3=np.zeros(Nthetas*(Nthetas+1)*(Nthetas+2)//6)
counter=0
for i in range(Nthetas):
    for j in range(i, Nthetas):
        for k in range(j, Nthetas):
            map3_mean=np.mean(Maps[i][mask]*Maps[j][mask]*Maps[k][mask])
            Map3[counter]=map3_mean
            counter+=1
            
np.savetxt(f"MapMapMap_fromCurved_Kappa_nside_{nside}.dat", Map3)

In [ ]:
# Plot <Map³>
initPlot(usetex=False)
fig, ax=plt.subplots()
prepareMap3Plot(ax)
ax.plot(Map3)
ax.set_ylabel(r"$\langle M_\mathrm{ap}^3 \rangle$")
finalizePlot(ax, title=r"$\langle M_\mathrm{ap}^3\rangle$ from curved sky "+f"nside={nside}", outputFn=f"MapMapMap_fromCurved_Kappa_nside_{nside}.png")